In [1]:
from pyomo.environ import *

In [2]:
# PU calcualtions
sbase = 100;
#Line impeadance PU
x12 = 0.0281   
x14 = 0.0304
x15 = 0.0064
x23 = 0.0108
x34 = x45 = 0.0297
#slack bus 
delta1 = 0        #ask q 

#rounding number
k = 3

In [3]:
#defining model
model = AbstractModel()
#Power Generation
model.pg1 = Var(bounds=(0,0.4))
model.pg2 = Var(bounds=(0,1.7))
model.pg3 = Var(bounds=(0,5.2))
model.pg4 = Var(bounds=(0,2))
model.pg5 = Var(bounds=(0,6))


#power being sent from from one bus to the next
model.p12 = Var(bounds=(-4,4))
model.p14 = Var(bounds=(-4,4))
model.p15 = Var(bounds=(-4,4))
model.p23 = Var(bounds=(-4,4))
model.p34 = Var(bounds=(-4,4))
model.p45 = Var(bounds=(-2.4,2.4))


#voltage angle @ each bus 
model.delta2 = Var()
model.delta3 = Var()
model.delta4 = Var()
model.delta5 = Var()

In [4]:
'''   Constraints    '''


#active flow calculation between each pair of connected buses
def con_rule1(model):
    return model.pg1 + model.pg2  == model.p12 + model.p14 + model.p15
model.con1 = Constraint(rule=con_rule1)


def con_rule2(model):
    return model.p12  + model.p23 ==  3
model.con2 = Constraint(rule=con_rule2)


def con_rule3(model):
    return model.pg3 == model.p23+3+model.p34
model.con3 = Constraint(rule=con_rule3)

def con_rule4(model):
    
    return model.pg4 + model.p34 + model.p14 == 4 + model.p45

model.con4 = Constraint(rule=con_rule4)

def con_rule5(model):
    
    return model.pg5 + model.p45 + model.p15 == 0 

model.con5 = Constraint(rule=con_rule5)


#nodal active power balance in each bus
def con_rule6(model):
    
    return model.p12 == (delta1 - model.delta2)/x12

model.con6 = Constraint(rule=con_rule6)

def con_rule7(model):
    return model.p14 == (delta1 - model.delta4)/x14
model.conf7 = Constraint(rule=con_rule7)

def con_rule8(model):
    return model.p15 == (delta1 - model.delta5)/x15
model.con8 = Constraint(rule=con_rule8)

def con_rule9(model):
    return model.p23 == (model.delta3 - model.delta2)/x23
model.con9 = Constraint(rule=con_rule9)

def con_rule10(model):
    return model.p34 == (model.delta3 - model.delta4)/x34
model.con10 = Constraint(rule=con_rule10)

def con_rule11(model):
    return model.p45 == (model.delta4 - model.delta5)/x45
model.con11 = Constraint(rule=con_rule11)

In [5]:
#Objective Function, trying to minimize the cost.
def obj_rule(model):
    return 14*model.pg1+15*model.pg2+30*model.pg3+40*model.pg4+10*model.pg5
model.OF = Objective(rule=obj_rule,sense=minimize)


In [6]:
opt = SolverFactory('glpk')
instance = model.create_instance()
results = opt.solve(instance) # solves and updates instance

In [12]:
print("DC 5 Bus PowerFlow")
print("Example 6.5 taken from Power System Optimization Modeling in GAMS")
print("Report")
print('Power Generated at Bus 1 = '+str(round(sbase*value(instance.pg1),k))+" MW")
print('Power Generated at Bus 2 = '+str(round(sbase*value(instance.pg2),k))+" MW")
print('Power Generated at Bus 3 = '+str(round(sbase*value(instance.pg3),k))+" MW")
print('Power Generated at Bus 4 = '+str(round(sbase*value(instance.pg4),k))+" MW")
print('Power Generated at Bus 5 = '+str(round(sbase*value(instance.pg5),k))+" MW")
print("--------------------------------------")
print('Power Flowing from bus 1 to 2 = '+str(round(sbase*value(instance.p12),k))+" MW")
print('Power Flowing from bus 1 to 4 = '+str(round(sbase*value(instance.p14),k))+" MW")
print('Power Flowing from bus 1 to 5 = '+str(round(sbase*value(instance.p15),k))+" MW")

print('Power Flowing from bus 2 to 3 = '+str(round(sbase*value(instance.p23),k))+" MW")
print('Power Flowing from bus 3 to 4 = '+str(round(sbase*value(instance.p34),k))+" MW")
print('Power Flowing from bus 4 to 5 = '+str(round(sbase*value(instance.p45),k))+" MW")
print("--------------------------------------")
print('Delta 1 = '+str(delta1))
print('Delta 2 = '+str(round(value(instance.delta2),k)))
print('Delta 3 = '+str(round(value(instance.delta3),k)))
print('Delta 4 = '+str(round(value(instance.delta4),k)))
print('Delta 5 = '+str(round(value(instance.delta5),k)))

print('cost = '+ str(round(sbase*value(instance.OF),k))+' €/Hour')

DC 5 Bus PowerFlow
Example 6.5 taken from Power System Optimization Modeling in GAMS
Report
Power Generated at Bus 1 = 40.0 MW
Power Generated at Bus 2 = 170.0 MW
Power Generated at Bus 3 = 323.495 MW
Power Generated at Bus 4 = 0.0 MW
Power Generated at Bus 5 = 466.505 MW
--------------------------------------
Power Flowing from bus 1 to 2 = 249.717 MW
Power Flowing from bus 1 to 4 = 186.788 MW
Power Flowing from bus 1 to 5 = -226.505 MW
Power Flowing from bus 2 to 3 = 50.283 MW
Power Flowing from bus 3 to 4 = -26.788 MW
Power Flowing from bus 4 to 5 = -240.0 MW
--------------------------------------
Delta 1 = 0
Delta 2 = -0.07
Delta 3 = -0.065
Delta 4 = -0.057
Delta 5 = 0.014
cost = 17479.897 €/Hour


In [8]:
print('Power Flowing from bus 1 to 2 = '+str(round(sbase*value(instance.p12),k))+" MW")
print('Power Flowing from bus 1 to 4 = '+str(round(sbase*value(instance.p14),k))+" MW")
print('Power Flowing from bus 1 to 5 = '+str(round(sbase*value(instance.p15),k))+" MW")

print('Power Flowing from bus 2 to 3 = '+str(round(sbase*value(instance.p23),k))+" MW")
print('Power Flowing from bus 3 to 4 = '+str(round(sbase*value(instance.p34),k))+" MW")
print('Power Flowing from bus 4 to 5 = '+str(round(sbase*value(instance.p45),k))+" MW")



Power Flowing from bus 1 to 2 = 249.717 MW
Power Flowing from bus 1 to 4 = 186.788 MW
Power Flowing from bus 1 to 5 = -226.505 MW
Power Flowing from bus 2 to 3 = 50.283 MW
Power Flowing from bus 3 to 4 = -26.788 MW
Power Flowing from bus 4 to 5 = -240.0 MW


In [9]:
print('Delta 1 = '+str(delta1))
print('Delta 2 = '+str(round(value(instance.delta2),k)))
print('Delta 3 = '+str(round(value(instance.delta3),k)))
print('Delta 4 = '+str(round(value(instance.delta4),k)))
print('Delta 5 = '+str(round(value(instance.delta5),k)))

Delta 1 = 0
Delta 2 = -0.07
Delta 3 = -0.065
Delta 4 = -0.057
Delta 5 = 0.014


In [10]:
print('cost = '+ str(round(sbase*value(instance.OF),k))+' €/Hour')

cost = 17479.897 €/Hour
